<div class='bar_title'></div>

*Data Driven Decisions in Practice (D3IP): Urban Analytics*

# Case Study: Predicting AirBnB Accomodation Prices

Gunther Gust & Nikolai Stein

Data Driven Decisions (D3) Group <br>
Center for Artificial Intelligence & Data Science <br>



<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

Complete the case study outlined by the steps below. Remember to always comment your code and document your findings so that your notebook is easy to read and follow! (Apart from correctness, the style of the notebook will also affect your grade!)



# PART 1: Data Loading and Exploratory Data Analysis

Insructions:

* Load the `airbnb.geojson` file into this notebook as a geodataframe
* Explore the content of each column of the geodataframe using methods of your choice (descriptive statistics, plots etc.). Describe your findings in the markdown cells.
* Create a plot that displays the location of the airbnb listings and the price. Add a basemap of San Diego to the plot.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt 
import seaborn as sns
import contextily as cx

In [ ]:
# Load the geodataframe
db = gpd.read_file("airbnb_listings.geojson")

#exploring the dataset...  
f, ax = plt.subplots(1, figsize=(16, 16))
db.plot(ax = ax, column = 'price', legend = True, alpha = 0.5, cmap = 'inferno', markersize = 10)
cx.add_basemap(ax, crs = db.crs, source=cx.providers.Esri.WorldImagery)
plt.show()                                                                                          #...by plotting the accommodations with their respective price

sns.histplot(db['price'])                                                                           #...alternatively as a histogram to visualize the distribution of the prices

db.describe()                                                                                       #...and using the describe function to look at the statistics of the geodataframe

Observations deducted from the shown plot, table and histogram:
- most of the AirBnB accomodations are relatively cheap (75% of the listings cost up to 250$) and are clustered on the lower end of the entire scale of given prices
- only a few outliers exist (the highest price of an accommodation is 4900$)
- the accommodations in the upper 25% of the price range are usually located near the coast or beach area
- there is a circular shaped cluster on the right that is hollowed out (Balboa Park is located their and the accommodations are clustered around it, hence this hollowed out ring like shape)
- the area called "La Jolla" contains most of the expensive accommodations of the upper 25% of the given price range 

These are the variables you should analyze and later use to predict the `price`:

In [ ]:

variable_names = [
    "accommodates",  # Number of people it accommodates
    "bathrooms",  # Number of bathrooms
    "bedrooms",  # Number of bedrooms
    "beds",  # Number of beds
    # Below are binary variables, 1 True, 0 False
    "rt_Private_room",  # Room type: private room
    "rt_Shared_room",  # Room type: shared room
    "pg_Condominium",  # Property group: condo
    "pg_House",  # Property group: house
    "pg_Other",  # Property group: other
    "pg_Townhouse",  # Property group: townhouse
]

# PART 2 Feature Engineering: Get points-of-interest (POIs) and prepare them

Instructions:

* Use the method `features_from_address()` of osmnx to download POIs of the type `amenity` (select the types of amenities to include from this list here: https://wiki.openstreetmap.org/wiki/Key:amenity)
* Pay attention to set the `dist` parameter to an appropiate value 

In [ ]:
import osmnx as ox

In [ ]:
# Download the POI data
#for plotting the POIs with all chosen amenity types included
pois = ox.features_from_address("San Diego, USA", dist = 5000, tags = {"amenity" : ["bar", "atm", "cafe", "restaurant"]})

#to calculate the KDE estimates for each POI amenity type separately
poiBars = ox.features_from_address("San Diego, USA", dist = 5000, tags = {"amenity" : "bar"})
poiATMs = ox.features_from_address("San Diego, USA", dist = 5000, tags = {"amenity" : "atm"})
poiCafes = ox.features_from_address("San Diego, USA", dist = 5000, tags = {"amenity" : "cafe"})
poiRestaurants = ox.features_from_address("San Diego, USA", dist = 5000, tags = {"amenity" : "restaurant"})


In [ ]:
pois.head()

Data cleaning: The resulting `pois` geodataframe may have a composite row index. In addition, some POIs may be of a strange element_type. You can use the following code to eliminate these issues.

In [ ]:
# Clean the POI data
pois.reset_index(inplace=True) # reset the index of the data frame
pois = pois[pois.element_type=="node"] # eliminate all POIs that are not of the type "node"


#Clean POI bei amenity
#Bar POI amenity type
poiBars.reset_index(inplace=True)
poiBars = poiBars[poiBars.element_type == "node"]


#ATM POI amenity type
poiATMs.reset_index(inplace=True)
poiATMs = poiATMs[poiATMs.element_type == "node"]


#Cafe POI amenity type
poiCafes.reset_index(inplace=True)
poiCafes = poiCafes[poiCafes.element_type == "node"]


#Restaurant POI amenity type
poiRestaurants.reset_index(inplace=True)
poiRestaurants = poiRestaurants[poiRestaurants.element_type == "node"]


Instructions:
* Plot the POIs spatially (use again a background map of San Diego)
* When you use POIs of different amenity types, color the POIs differently

In [ ]:
fAllPOIs, ax = plt.subplots(1)
pois.plot(alpha = 1, ax = ax, edgecolor = 'black', column = 'amenity', legend = True, cmap = 'plasma')
cx.add_basemap(ax, crs = db.crs)
ax.set_title("Different amenity types in San Diego")
ax.set_axis_off()
plt.show()

For each POI amenity type, create a kernel density estimation (KDE):
* Convert the `geometry` of the POI into a suitable coordinate data format (you may use the provided function `create_coordinate_array` for this)
* Feed the resulting coordinates into the `gaussian_kde` function and estimate the function
* Also convert the `geometry` of the Airbnb listings into the coordinate data format (you may use the provided function `create_coordinate_array` for this)
* Using the converted Airbnb geometries, compute the KDE for the locations of the Airbnb listings
* Add the KDE estimate as additional columns to your original airbnb geodataframe

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
# helper function to convert the geometries into a suitable coordinate format for the KDE
def create_coordinate_array(geometries): 
    x_values = []
    y_values = []

# Iterate through each row in the GeoDataFrame
    for multipoint in geometries:
        # Ensure the geometry is indeed MultiPoint; if it's just a single Point, wrap it in a list
        points = list(multipoint.geoms) if hasattr(multipoint, "geoms") else [multipoint]
        
        # For each Point in the MultiPoint, extract x and y values
        for point in points:
            x_values.append(point.x)
            y_values.append(point.y)

    # Optionally, convert the lists to numpy arrays for further processing
    x_values = np.array(x_values)
    y_values = np.array(y_values)

    # Rearrange data to create a 2D array of x and y coordinates
    xy = np.vstack([x_values,y_values])

    return xy


# Example Usage for the Airbnb geodataframe
#airbnb_array = create_coordinate_array(airbnb.geometry)

In [ ]:
#calculating KDE estimates with all POI amenity types included and for each amenity type individually for plotting

#WARNING#
#we ran into a ValueError when trying to add the calculated densities to the dataframe
#to not crash the notebook when executing, the problematic lines have been commented-out because we were unable to find the source of this error 

# Create coordinate array
xyDB = create_coordinate_array(db.geometry)
xyBars = create_coordinate_array(poiBars.geometry)
xyATMs = create_coordinate_array(poiBars.geometry)
xyCafes = create_coordinate_array(poiCafes.geometry)
xyRestaurants = create_coordinate_array(poiRestaurants.geometry)


# Compute the density estimation
kdeDB = gaussian_kde(xyDB)
kdeBars = gaussian_kde(xyBars)
kdeATMs = gaussian_kde(xyATMs)
kdeCafes = gaussian_kde(xyCafes)
kdeRestaurants = gaussian_kde(xyRestaurants)


# Defining the grid points 
xminDB, xmaxDB = xyDB[0].min(), xyDB[0].max()
yminDB, ymaxDB = xyDB[1].min(), xyDB[1].max()

xminBars, xmaxBars = xyBars[0].min(), xyBars[0].max()
yminBars, ymaxBars = xyBars[1].min(), xyBars[1].max()

xminATMs, xmaxATMs = xyATMs[0].min(), xyATMs[0].max()
yminATMs, ymaxATMs = xyATMs[1].min(), xyATMs[1].max()

xminCafes, xmaxCafes = xyCafes[0].min(), xyCafes[0].max()
yminCafes, ymaxCafes = xyCafes[1].min(), xyCafes[1].max()

xminRestaurants, xmaxRestaurants = xyRestaurants[0].min(), xyRestaurants[0].max()
yminRestaurants, ymaxRestaurants = xyRestaurants[1].min(), xyRestaurants[1].max()

#create 100 evenly spaced points between min and max
xxDB, yyDB = np.mgrid[xminDB:xmaxDB:100j, yminDB:ymaxDB:100j]
xxBars, yyBars = np.mgrid[xminBars:xmaxBars:100j, yminBars:ymaxBars:100j]
xxATMs, yyATMs = np.mgrid[xminATMs:xmaxATMs:100j, yminATMs:ymaxATMs:100j]
xxCafes, yyCafes = np.mgrid[xminCafes:xmaxCafes:100j, yminCafes:ymaxCafes:100j]
xxRestaurants, yyRestaurants = np.mgrid[xminRestaurants:xmaxRestaurants:100j, yminRestaurants:ymaxRestaurants:100j]

# Evaluate the density at grid points
densityDB = kdeDB(np.vstack([xxDB.ravel(), yyDB.ravel()]))
densityDB = densityDB.reshape(xxDB.shape) # reshape to the original shape of xx (for plotting)
#db['KDE estimate of all POIs'] = densityDB

densityBars = kdeBars(np.vstack([xxBars.ravel(), yyBars.ravel()]))
densityBars = densityBars.reshape(xxBars.shape) # reshape to the original shape of xx (for plotting)
#db['KDE estimate of POI amenity type: Bar'] = densityBars

densityATMs = kdeATMs(np.vstack([xxATMs.ravel(), yyATMs.ravel()]))
densityATMs = densityATMs.reshape(xxATMs.shape) # reshape to the original shape of xx (for plotting)
#db['KDE estimate of POI amenity type: ATM'] = densityATMs

densityCafes = kdeCafes(np.vstack([xxCafes.ravel(), yyCafes.ravel()]))
densityCafes = densityCafes.reshape(xxCafes.shape) # reshape to the original shape of xx (for plotting)
#db['KDE estimate of POI amenity type: Cafe'] = densityCafes

densityRestaurants = kdeRestaurants(np.vstack([xxRestaurants.ravel(), yyRestaurants.ravel()]))
densityRestaurants = densityRestaurants.reshape(xxRestaurants.shape) # reshape to the original shape of xx (for plotting)
#db['KDE estimate of POI amenity type: Restaurant'] = densityRestaurants

Create a spatial point plot(s) of the Airbnb listings and color the points according to the KDE estimates, in order to check your results.

In [ ]:
plt.imshow(np.rot90(densityDB), cmap='hot', extent=[xminDB, xmaxDB, yminDB, ymaxDB])
plt.colorbar(label='Density')
plt.scatter(xyDB[0], xyDB[1], s=10, c='blue', alpha=0.5)
plt.title('Density of the AirBnB accommodations')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

plt.imshow(np.rot90(densityBars), cmap='hot', extent=[xminBars, xmaxBars, yminBars, ymaxBars])
plt.colorbar(label='Density')
plt.scatter(xyBars[0], xyBars[1], s=10, c='blue', alpha=0.5)
plt.title('KDE for POI amenity type: Bar')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

plt.imshow(np.rot90(densityATMs), cmap='hot', extent=[xminATMs, xmaxATMs, yminATMs, ymaxATMs])
plt.colorbar(label='Density')
plt.scatter(xyATMs[0], xyATMs[1], s=10, c='blue', alpha=0.5)
plt.title('KDE for POI amenity type: ATM')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

plt.imshow(np.rot90(densityCafes), cmap='hot', extent=[xminCafes, xmaxCafes, yminCafes, ymaxCafes])
plt.colorbar(label='Density')
plt.scatter(xyCafes[0], xyCafes[1], s=10, c='blue', alpha=0.5)
plt.title('KDE for POI amenity type: Cafe')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

plt.imshow(np.rot90(densityRestaurants), cmap='hot', extent=[xminRestaurants, xmaxRestaurants, yminRestaurants, ymaxRestaurants])
plt.colorbar(label='Density')
plt.scatter(xyRestaurants[0], xyRestaurants[1], s=10, c='blue', alpha=0.5)
plt.title('KDE for POI amenity type: Restaurant')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# PART 3: Modeling and Analysis (aka Machine Learning)

Use your dataset generated in the previous steps to predict AirBnb prices

Instructions:

* Split your data set into training and validation data sets
* Define an error metric (or several)
* Train at least one machine learning model (e.g. random forest) 
* Tune the hyperparameters (if applicable for the model)
* Evaluate the accuracy of the predicted prices against actual prices
* Compare the performance of the previous models when using different input data sets (benchmarks). Make sure to include the naive benchmark of predicting always the mean price.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor

#removing missing values
db.dropna(axis=0, inplace=True)

#prediction target
y = db['price']

#features
#in the following lines we tested different feature inputs and there effects on the mae (e.g. with or without property group)
db_features = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'rt_Private_room', 'rt_Shared_room', 'pg_Condominium', 'pg_House', 'pg_Other', 'pg_Townhouse']
#db_features = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'pg_Condominium', 'pg_House', 'pg_Other', 'pg_Townhouse']
#db_features = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'rt_Private_room', 'rt_Shared_room']
#db_features = ['accommodates', 'bathrooms', 'bedrooms', 'beds']
#db_features = ['accommodates', 'bathrooms', 'beds']

X = db[db_features]

#split data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

#Funktion für mae vergleich
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)




Removing all property group variables had no impact on the results.

Removing all room type variables improved the mae score. 

Removing all property group variables and all room type variables achieved a worse mae score.

Removing numeric variables has a much higher negative impact on the mae score than removing binary variables.

## Model 1: Decision Tree

In [ ]:
#define and fit model
model_one = DecisionTreeRegressor(random_state=1) #durch die 1 immer gleiche Ergebnisse
model_one.fit(train_X, train_y)

#predict and mae 
val_predictions = model_one.predict(val_X)
val_mae = mean_absolute_error(val_y, val_predictions)

#mae vergleich zwischen in-sample und out-of-sample mithilfe von Funktion
for max_leaf_nodes in [2, 5, 50, 500, 5000, 10000]:
    is_mae = get_mae(max_leaf_nodes, X, X, y, y)
    oos_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t In-sample:  %d \t Out-of-sample:  %d" %(max_leaf_nodes, is_mae, oos_mae))



The decision tree model is just a basic model to get a view first mae score samples.
Not very effective to achieve the best mae score.

## Model 2: Random Forest

In [ ]:
#test lists
n_list = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
m_list = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

y = 10
print("")
print("varying n_estimator to find best mae score")
for i in n_list:
    
    # Define and fit
    forest_model = RandomForestRegressor(random_state=1, n_estimators=i, max_depth=y) #n_estimators = Anzahl Bäume, max_depth = max Tiefe Bäume (Gefahr Overfitting)
    forest_model.fit(train_X, train_y)
    # Evaluatio
    forest_predictions = forest_model.predict(val_X) 
    print("The MAE of our model with a n_estimator of {} and a max depth of {} is: {}".format(i, y, mean_absolute_error(val_y, forest_predictions)))

x = 100
print("")
print("varying max_depth to find best mae score")
for j in m_list:        
    
    # Define and fit
    forest_model = RandomForestRegressor(random_state=1, n_estimators=x, max_depth=j) #n_estimators = Anzahl Bäume, max_depth = max Tiefe Bäume (Gefahr Overfitting)
    forest_model.fit(train_X, train_y)
    # Evaluatio
    forest_predictions = forest_model.predict(val_X) 
    print("The MAE of our model with a n_estimator of {} and a max depth of {} is: {}".format(x, j, mean_absolute_error(val_y, forest_predictions)))

print("")

forest_model = RandomForestRegressor(random_state=1, n_estimators=150, max_depth=8) #n_estimators = Anzahl Bäume, max_depth = max Tiefe Bäume (Gefahr Overfitting)
forest_model.fit(train_X, train_y)
#Evaluatio
forest_predictions = forest_model.predict(val_X) 
print("The best MAE of our model with a n_estimator of 150 and a max depth of 8 is: {}".format(mean_absolute_error(val_y, forest_predictions)))



The random forest model uses many trees and makes a prediction by the average. It has a much better accuracy than the previous model.

You can adjust many different hyperparameters. For test purposes we played around with two of them.
First the n_estimators (number of trees) and second the max_depth (max depth of the trees).

Our tests revealed following results:

The impact of different n_estimators is very low. 
150 achieves the best mae score

A higher max_depth causes overfitting.
a lower max_depth improved the mae. 
8 achieves the best mae score

best mae: 94.9671



## Model 3: Linear Regression

In [ ]:
# Define and fit
linear_model = LinearRegression() #keine Hyperparameter
linear_model.fit(train_X, train_y)

#Evaluate
linear_predictions = linear_model.predict(val_X)
print("The MAE of our model is: {}".format(mean_absolute_error(val_y, linear_predictions)))

From the linear regression model we get only one result because we dont have any tunable hyperparameters.

best mae: 104.3475

## Model 4: Huber Regressor 

In [ ]:
#less sensitive to outliers

#test list
epsilon_list = [1.00, 1.05, 1.10, 1.15, 1.20, 1.25, 1.30, 1.35, 1.40, 1.45, 1.50, 2.0, 3.0, 5.0, 10.0]

for i in epsilon_list:
    # Define and fit
    huber_model = HuberRegressor(epsilon=i) #epsilon = Steuert Robustheit
    huber_model.fit(train_X, train_y)

    #Evaluate
    huber_predictions = huber_model.predict(val_X)
    print("The MAE of our model using an epsilon of {} is: {}".format(i, mean_absolute_error(val_y, huber_predictions)))

A big advantage of the huber regressor is that this model is less sensitive to outliers. In our example we have less very expensive airbnbs which are outliers. 
Because of this the huber regressor is a very good model for our example and our tests.

Like the random forest model this model also has many adjustable hyperparameters.
For the tests we chose the hyperparameter epsilon which adjusts the robustness.

Our tests revealed following results:

a lower epsilon value improved the mae.
1.0 achieves the best mae score.

best mae: 94.6993